<h2 style="border:2px solid purple; border-radius:10px; color:white; background-color:green"><center> Tabular Playground Series(feb) </center></h2>

Table Playground Series are beginner friendly monthly competitions organised by kaggle.<br/>
    
In this competition we have to make a regrssion model based on categorical and continous features provided<br/>

This notebook is beginner friendly guide for creating supercool EDA and making baseline model.
    
**Feel free to ask any question and if you find any mistake tell me in the comments**

## Importing Libraries 📗.

In [ ]:
import os
import gc
import sys
import time
import tqdm
import random
import numpy as np
import pandas as pd
import seaborn as sns
import datatable as dt
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff

from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

from sklearn.manifold import TSNE

from colorama import Fore, Back, Style
y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings('ignore')

import lightgbm as lgb

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset

from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import (StandardScaler,PowerTransformer,
                                   QuantileTransformer,LabelEncoder, 
                                   OneHotEncoder, OrdinalEncoder,
                                  RobustScaler)

<h2 style="border:2px solid purple; border-radius:10px; color:black; background-color:yellow"><center> 1. Given Data 💽 </center></h2>

we are provided with 3 files.<br/>
1. sample_submission.csv
2. test.csv
3. train.csv

Train data contains 10 categorical featues and 14 continous features<br/>
we have to make prediction on test data and make submission using sample_submission's format<br/>

<h2 style="border:2px solid purple; border-radius:10px; color:black; background-color:yellow"><center> 2. Metric 📐</center></h2>

Here metric used is very simple which is root mean squared error.<br/>

rmse = root(1/n (yit-yip)^2)

Here yit is true target values<br/>
and yip is predicted target values.<br/>

<h2 style="border:2px solid purple; border-radius:10px; color:black; background-color:yellow"><center> 3. Loading Data 💽</center></h2>

We will use pandas to load data<br/>
 

In [ ]:
folder_path = '../input/tabular-playground-series-feb-2021'
train_data = pd.read_csv(f'{folder_path}/train.csv')
test_data = pd.read_csv(f'{folder_path}/test.csv')
sample = pd.read_csv(f'{folder_path}/sample_submission.csv')

In [ ]:
print("{0}Number of rows in train data: {1}{2}\n{0}Number of columns in train data: {1}{3}".format(y_,r_,train_data.shape[0],train_data.shape[1]))
print("{0}Number of rows in test data: {1}{2}\n{0}Number of columns in test data: {1}{3}".format(m_,r_,test_data.shape[0],test_data.shape[1]))
print("{0}Number of rows in sample : {1}{2}\n{0}Number of columns in sample : {1}{3}".format(c_,r_,sample.shape[0],sample.shape[1]))

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
sample.head()

<h2 style="border:2px solid purple; border-radius:10px; color:black; background-color:yellow"><center> 4. Exploratory Data Analysis 📊📈📉👀</center></h2>

In [ ]:
cont_features = [f'cont{i}' for i in range(14)]
cat_features = [f'cat{i}' for i in range(10)]
all_features = cont_features + cat_features

<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center> 4.1 Checking for Null Values
</center></h3>

In [ ]:
train_data.isnull().sum().sum()

<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center>4.2 Distribution of cont0</center></h3>

In [ ]:
plt.style.use('fivethirtyeight')
def distribution1(feature,color1,color2,df=train_data):
    plt.figure(figsize=(15,7))
    
    plt.subplot(121)
    dist = sns.distplot(df[feature],color=color1)
    a = dist.patches
    xy = [(a[i].get_x() + a[i].get_width() / 2,a[i].get_height()) \
          for i in range(1,len(a)-1) if (a[i].get_height() > a[i-1].get_height() and a[i].get_height() > a[i+1].get_height())]
    
    for i,j in xy:
        dist.annotate(
            s=f"{i:.3f}",
            xy=(i,j), 
            xycoords='data',
            ha='center', 
            va='center', 
            fontsize=11, 
            color='black',
            xytext=(0,7), 
            textcoords='offset points',
        )
    
    qnt = df[feature].quantile([.25, .5, .75]).reset_index(level=0).to_numpy()
    plt.subplot(122)
    box = sns.boxplot(df[feature],color=color2)
    for i,j in qnt:
        box.annotate(str(j)[:4],xy= (j-.05,-0.01),horizontalalignment='center')
        
    print("{}Max value of {} is: {} {:.2f} \n{}Min value of {} is: {} {:.2f}\n{}Mean of {} is: {}{:.2f}\n{}Standard Deviation of {} is:{}{:.2f}"\
      .format(y_,feature,r_,df[feature].max(),g_,feature,r_,df[feature].min(),b_,feature,r_,df[feature].mean(),m_,feature,r_,df[feature].std()))

In [ ]:
distribution1('cont0','yellow','red')

<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center> 4.3 Distribution of targets</center></h3>

In [ ]:
distribution1('target','red','blue')

Target ranges from 0 to 10.31<br/>
but distribution really starts from target values >4<br/>

We can see 2 peaks in target distribution<br/>
One may assume that data is taken from 2 different distribution and then combined.<br/>
so we are seeing two peaks<br/>

But judging from TPS(jan) actually target is from same distribution but some values of targets<br/>
belong to test data so we will most probably see the distribution of predictions with one peak around value 7<br/>

<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center> 4.3 Distribution of all the continous features. </center></h3>

In [ ]:
plt.figure(figsize=(20,15))
colors = ['#8ECAE6','#219EBC','#023047',
          '#023047','#023047','#0E402D',
          '#023047','#023047','#F77F00',
          '#D62828','#4285F4','#EA4335',
          '#FBBC05','#34A853']
for i,feature in enumerate(cont_features):
    plt.subplot(2,7,i+1)
    sns.distplot(train_data[feature],color=colors[i])

It is clear that all the features have different disbtributions and none of them looks like normal distribution<br/>

cont1 is very interesting as there are gaps in cont1 as if it is partly continous and partly categorical<br/>

Maybe we should look into correlations between this features to get better insight

<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center> 4.4 Correlation Matrix</center></h3>

In [ ]:
corr = train_data[all_features+['target']].corr()
fig = px.imshow(corr)
fig.show()

We see that there is some correlation between      featues cont5 to cont12 but they are not very high.
    
One interesting thing is no features have any    correlation with target

<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center>4.5 Pairplot of features</center></h3>

In [ ]:
def plot_grid(data,color1,color2,color3):
    f = sns.PairGrid(data);
    plt.figure(figsize=(10,10));
    f.map_upper(plt.scatter,color = color1);
    f.map_lower(sns.kdeplot,color = color2);
    #f.map_diag(sns.histplot,color = c3 );
    f.map_diag(sns.kdeplot, lw=3, legend=False,color = color3);

In [ ]:
plot_grid(train_data.loc[:1000,all_features+['target']],'#EA4335','#FBBC05','#34A853');

Pair plot gives very good idea about relationship between the features<br/>
all the features have a square relationship means for every value in one feature<br/>
there are enough values in entire range of other feature which makes this data little challanging<br/>

<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center> 4.7 Let's play with PCA</center></h3>

<h4 style="border:2px solid purple; border-radius:10px; color:black; background-color:lightblue"><center> 4.7.1 How good are 4 components of pca at seprating data</center></h4>

In [ ]:
def pca_plot1(features,n_components,target,nrows=10**4):
    pca = PCA(n_components=n_components)
    train_d = train_data.sample(n=nrows).fillna(train_data.mean())
    train_g_pca = pca.fit_transform(train_d[features])

    total_var = pca.explained_variance_ratio_.sum()*100
    labels = {str(i): f"PC {i+1}" for i in range(n_components)}

    fig = px.scatter_matrix(
        train_g_pca,
        dimensions=range(n_components),
        labels=labels,
        title=f"Total explained variance ratio{total_var:.2f}%",
        color=train_d[target].values
    )

    fig.update_traces(diagonal_visible=True,opacity=0.5)
    fig.show()

In [ ]:
pca_plot1(cont_features,4,'target')

<h4 style="border:2px solid purple; border-radius:10px; color:black; background-color:lightblue"><center> 4.7.2 How good are 3 components of pca at seprating data (3d plot)</center></h4>

In [ ]:
def pca_plot_3d(features,target,nrows=10**4):
    pca = PCA(n_components=3)
    train_d = train_data.sample(n=nrows).fillna(train_data.mean())
    train_g_pca = pca.fit_transform(train_d[features])

    total_var = pca.explained_variance_ratio_.sum()*100

    fig = px.scatter_3d(
        train_g_pca,x=0,y=1,z=2,
        title=f"Total explained variance ratio{total_var:.2f}%",
        color=train_d[target].values,
        labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )

    fig.show()

In [ ]:
pca_plot_3d(cont_features,'target')

<h4 style="border:2px solid purple; border-radius:10px; color:black; background-color:lightblue"><center> 4.7.3 Ploting explained variance </center></h4>

In [ ]:
def plot_exp_var(features,nrows=10**4):
    pca = PCA()
    train_d = train_data.sample(n=nrows).fillna(train_data.mean())
    pca.fit(train_d[features])
    exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

    fig = px.area(
        x=range(1, exp_var_cumul.shape[0] + 1),
        y=exp_var_cumul,
        labels={"x": "# Components", "y": "Explained Variance"},
    )
    fig.show()

In [ ]:
plot_exp_var(cont_features)

<h4 style="border:2px solid purple; border-radius:10px; color:black; background-color:lightblue"><center> 4.8 Countplot of all categorical fearues </center></h4>

In [ ]:
plt.style.use("ggplot")
plt.figure(figsize=(25,20))
for i,feature in enumerate(cat_features):
    plt.subplot(2,5,i+1)
    sns.countplot(train_data[feature])

It is clear that distribution of categories are evry skewed<br/>
It would be good to remove some of the cat features like<br/>
cat4,cat0, cat2, cat6, cat7, <br/>
The reason for this is that almost all rows will have same values for these columns<br/>
so it will not provide any usefull information to model.

 <h2 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center> 4.9 cont0 distribution based on cat9</center></h2>
As cat9 has most categories let us see distribution of one feature based on cat9.

In [ ]:
def distribution3(feature,category,df=train_data):
    plt.subplots(figsize=(15, 7))
    sns.histplot(train_data,x=feature,hue=category)

In [ ]:
distribution3('cont0','cat9')

<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center> 4.10 cont0 box-plot based on cat9</center></h3>
I think box-plot will give use better idea than histplot

In [ ]:
def boxploting1(feature,category,df=train_data,figure_size=(15,7)):
    plt.subplots(figsize=figure_size)
    sns.boxplot(x=feature, y=category, data=df,whis=[0, 100], width=.6, palette="vlag")

In [ ]:
boxploting1('cont0','cat9')

<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center> 4.11 Boxenplot for cont1 and cat8
 </center></h3>

In [ ]:
def boxploting2(feature,category,df=train_data,figure_size=(15,7)):
    plt.subplots(figsize=figure_size)
    sns.boxenplot(y=feature, x=category,color="pink",scale="linear", data=df)

In [ ]:
boxploting2('cont0','cat8',figure_size=(10,7))

<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center> 4.12 Trivariate histogram with 2 category</center></h3>

In [ ]:
def distribution3(feature,category1,category2,df=train_data,figure_size=(15,15)):
    sns.set_theme(style="dark")
    sns.displot(
        data=df, x=feature, y=category1, col=category2,
        log_scale=(True, False), col_wrap=4, height=4, aspect=.7,
    )

In [ ]:
distribution3('cont5','cat8','cat9')

<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center> 4.13 swarmplot of target</center></h3>

In [ ]:
def swarmplot(feature,category1,category2,df=train_data,figure_size=(15,7)):
    sns.set_theme(style="whitegrid", palette="muted")
    plt.figure(figsize=figure_size)
    ax = sns.swarmplot(data=df, x=feature, y=category1, hue=category2)
    ax.set(ylabel="")

In [ ]:
swarmplot('target','cat8','cat9',df=train_data.sample(n=10000))

<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center>4.13 Trend lines and templates plotly</center></h3>
Now I will use plotly for plotting

In [ ]:
def scatterplot1(feature1,feature2,category,df=train_data):
    fig = px.scatter(train_data, x=feature1, y=feature2, color=category, marginal_y="violin",
               marginal_x="box", trendline="ols", template="simple_white")
    fig.show()

In [ ]:
scatterplot1('cont6','cont7','cat1')

<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center> 4.14 Error bar </center></h3>

In [ ]:
def errorbar(feature1,feature2,feature3,category,df=train_data):
    df['e'] = df[feature3]/100
    fig = px.scatter(df, x=feature1, y=feature2, color=category, error_x="e", error_y="e")
    fig.show()

In [ ]:
errorbar('cont1','cont2','cont3','cat9',df=train_data.sample(n=1000))

<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center> 4.15 parellel categories</center></h3>

In [ ]:
def parallelcat(feature,df=train_data):
    fig = px.parallel_categories(df, color=feature, color_continuous_scale=px.colors.sequential.Inferno)
    fig.show()

In [ ]:
parallelcat('cont2',df=train_data.sample(n=1000))


<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center> 4.16 Parallel lines</center></h3>

In [ ]:
def parallellines(feature,df=train_data):
    fig = px.parallel_coordinates(df, color=feature,
                    color_continuous_scale=px.colors.diverging.Tealrose, color_continuous_midpoint=2)
    fig.show()

In [ ]:
parallellines('target',df=train_data.sample(n=1000))

<h3 style="border:2px solid purple; border-radius:10px; color:black; background-color:pink"><center> 4.17 sunburst Chart</center></h3>

In [ ]:
def sunburst(category1,category2,df=train_data):
    fig = px.sunburst(df, path=[category1, category2],
                  color='target')
    fig.show()

In [ ]:
sunburst('cat8','cat9',df=train_data.sample(n=1000))

<h2 style="border:2px solid purple; border-radius:10px; color:black; background-color:brown"><center> 5. Data Preprocessing</center></h2>
I am using StratifiedKFold for regression by dividing target into bins


In [ ]:
folder_path = '../input/tabular-playground-series-feb-2021'
train_data = pd.read_csv(f'{folder_path}/train.csv')
test_data = pd.read_csv(f'{folder_path}/test.csv')
sample = pd.read_csv(f'{folder_path}/sample_submission.csv')

cont_features = [f'cont{i}' for i in range(14)]
cat_features = [f'cat{i}' for i in range(10)]
# cat_features = ['cat1','cat3','cat5','cat8','cat9']
all_features =   cat_features + cont_features
target_feature = 'target'

num_bins = int(1 + np.log2(len(train_data)))
train_data.loc[:,'bins'] = pd.cut(train_data['target'].to_numpy(),bins=num_bins,labels=False)
bins = train_data['bins'].to_numpy()

for feat in cat_features:
    le = LabelEncoder()
    train_data.loc[:,feat] = le.fit_transform(train_data[feat].fillna("-1"))
    test_data.loc[:,feat] = le.transform(test_data[feat].fillna("-1"))


qt = QuantileTransformer()
train_data.loc[:,cont_features] = qt.fit_transform(train_data.loc[:,cont_features])
test_data.loc[:,cont_features] = qt.transform(test_data.loc[:,cont_features])

emb_c = {cat: int(train_data[cat].nunique()) for cat in cat_features if int(train_data[cat].nunique()) >2}
emb_cols = emb_c.keys()
embedding_sizes = [(c, min(50, (c+1)//2)) for _,c in emb_c.items()]
cont_features = cont_features + [cat for cat in cat_features if cat not in emb_cols]
cat_features = emb_cols

# target = train_data[target_feature].to_numpy()
# train_data = train_data[all_features].to_numpy()
# test_data = test_data[all_features].to_numpy()

train_data.shape, test_data.shape

<h2 style="border:2px solid red; border-radius:10px; color:black; background-color:orange"><center> 6. Pytorch Model</center></h2>

In [ ]:
def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))

In [ ]:
seed = 2021

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=seed)

In [ ]:
config = {
    "epochs":15,
    "train_batch_size":1024,
    "valid_batch_size":1024,
    "test_batch_size":1024,
    "nfolds":5, 
    "learning_rate":0.001,
    
    "input_size":len(all_features), 
    'cont_size':len(cont_features),
    'hidden_sizes':[128,64,32,16],
    'output_size':1
}

In [ ]:
class TPSDataset(Dataset):
    def __init__(self,df,cat_features,cont_features):
        self.cat_data = df.loc[:,cat_features].to_numpy()
        self.cont_data = df.loc[:,cont_features].to_numpy()
        self.target = df.loc[:,target_feature].to_numpy()
    
    def __getitem__(self,idx):
        input1 = torch.tensor(self.cat_data[idx],dtype=torch.long)
        input2 = torch.tensor(self.cont_data[idx],dtype=torch.long)
        target = torch.tensor(self.target[idx],dtype=torch.float)
        return input1,input2, target
    
    def __len__(self):
        return len(self.target)

In [ ]:
class Model(nn.Module):
    def __init__(self,cont_size,output_size,hidden_sizes,embedding_sizes):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(cat,size) for (cat,size) in embedding_sizes])
        self.emb_size = sum(e.embedding_dim for e in self.embeddings)
        self.cont_size = cont_size
            
        self.layer1 = self.batch_linear_drop(self.emb_size+self.cont_size,hidden_sizes[0],0.1,activation=nn.ReLU)
        self.layer2 = self.batch_linear_drop(hidden_sizes[0],hidden_sizes[1],0.1,activation=nn.ReLU)
        self.layer3 = self.batch_linear_drop(hidden_sizes[1],hidden_sizes[2],0.1,activation=nn.ReLU)
        self.layer4 = self.batch_linear_drop(hidden_sizes[2],hidden_sizes[3],0.1,activation=nn.ReLU)
        self.layer5 = self.batch_linear(hidden_sizes[3],output_size)
        
    def batch_linear_drop(self,inp,out,drop,activation=None):
        if activation:
            return nn.Sequential(nn.BatchNorm1d(inp),nn.Dropout(drop),nn.Linear(inp,out),activation())
        else:
            return nn.Sequential(nn.BatchNorm1d(inp),nn.Dropout(drop),nn.Linear(inp,out))
            
    def batch_linear(self,inp,out,activation=None):
        if activation:
            return nn.Sequential(nn.BatchNorm1d(inp),nn.Linear(inp,out),activation())
        else:
            return nn.Sequential(nn.BatchNorm1d(inp),nn.Linear(inp,out))
    
    def forward(self,input1,input2):
        x = [e(input1[:,i]) for i,e in enumerate(self.embeddings)]
        x = torch.cat(x,1)
        x = torch.cat([x,input2],1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        return x

In [ ]:
def run(plot_losses=True,verbose=True):
    
    def loss_fn(outputs,targets):
        loss = nn.MSELoss()(outputs,targets)
        return loss
  
    def train_loop(train_loader,model,loss_fn,device,optimizer,lr_scheduler=None):
        model.train()
        total_loss = 0
        for i, (inputs1,inputs2, targets) in enumerate(train_loader):
            inputs1 = inputs1.to(device)
            inputs2 = inputs2.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs1,inputs2)
            loss = loss_fn(outputs,targets)
            loss.backward()
                
            total_loss += loss.item()

            optimizer.step()
                    
        total_loss /= len(train_loader)
        return total_loss
    
    def valid_loop(valid_loader,model,loss_fn,device):
        model.eval()
        total_loss = 0
        predictions = list()
        
        for i, (inputs1,inputs2,targets) in enumerate(valid_loader):
            inputs1 = inputs1.to(device)
            inputs2 = inputs2.to(device)
            targets = targets.to(device)
            
            outputs = model(inputs1,inputs2)
            loss = loss_fn(outputs,targets)

            predictions.extend(outputs.detach().cpu().numpy())
            
            total_loss += loss.item()
        total_loss /= len(valid_loader)
            
        return total_loss,np.array(predictions)
    
    
    fold_train_losses = list()
    fold_valid_losses = list()
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"{device} is used")
    
#     kfold = KFold(n_splits=config['nfolds'])
    kfold = StratifiedKFold(n_splits=config['nfolds'])
    for k , (train_idx,valid_idx) in enumerate(kfold.split(X=train_data,y=bins)):
        x_train,x_valid = train_data.iloc[train_idx,:],train_data.iloc[valid_idx,:]

        input_size = config['cont_size']
        hiddens_sizes = config['hidden_sizes']
        output_size = 1

        model = Model(input_size,output_size,hiddens_sizes,embedding_sizes)
        model.to(device)

        train_ds = TPSDataset(x_train,cat_features,cont_features)
        train_dl = DataLoader(train_ds,
                             batch_size = config["train_batch_size"],
                              shuffle=True,
                              num_workers = 4,
                              pin_memory=True
                             )

        valid_ds = TPSDataset(x_valid,cat_features,cont_features)
        valid_dl = DataLoader(valid_ds,
                              batch_size =config["valid_batch_size"],
                              shuffle=False,
                              num_workers = 4,
                              pin_memory=True,
                             )
        
        optimizer = optim.Adam(model.parameters(),lr=config['learning_rate'])
        lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=2, verbose=True)
#         lr_scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.8)
#         lr_scheduler = None

        print(f"Fold {k}")
        best_loss = 999
        
        train_losses = list()
        valid_losses = list()
        start = time.time()
        for i in range(config["epochs"]):
            train_loss = train_loop(train_dl,model,loss_fn,device,optimizer,lr_scheduler=lr_scheduler)
            valid_loss,predictions = valid_loop(valid_dl,model,loss_fn,device)
            
            if lr_scheduler:
                lr_scheduler.step(valid_loss)

            train_losses.append(train_loss)
            valid_losses.append(valid_loss)
            
            end = time.time()
            epoch_time = end - start
            start = end
            
            score = rmse_score(x_valid[target_feature],predictions)
                          
            if verbose:
                print(f"epoch:{i} Training loss:{train_loss} | Validation loss:{valid_loss}| Score: {score}  | epoch time {epoch_time:.2f}s ")

            if valid_loss <= best_loss:
                if verbose:
                    print(f"{g_}Validation loss Decreased from {best_loss} to {valid_loss}{sr_}")
                best_loss = valid_loss
                torch.save(model.state_dict(),f'model{k}.bin')
                
        fold_train_losses.append(train_losses)
        fold_valid_losses.append(valid_losses)
#         break
      
        
    if plot_losses == True:
        plt.figure(figsize=(20,14))
        for i, (t,v) in enumerate(zip(fold_train_losses,fold_valid_losses)):
            plt.subplot(2,5,i+1)
            plt.title(f"Fold {i}")
            plt.plot(t,label="train_loss")
            plt.plot(v,label="valid_loss")
            plt.legend()
        plt.show()

In [ ]:
run()

<h2 style="border:2px solid red; border-radius:10px; color:white; background-color:orange"><center> Inference </center></h2>

In [ ]:
class TPSDataset(Dataset):
    def __init__(self,df,cat_features,cont_features):
        self.cat_data = df.loc[:,cat_features].to_numpy()
        self.cont_data = df.loc[:,cont_features].to_numpy()
    
    def __getitem__(self,idx):
        input1 = torch.tensor(self.cat_data[idx],dtype=torch.long)
        input2 = torch.tensor(self.cont_data[idx],dtype=torch.long)
        return input1,input2
    
    def __len__(self):
        return len(self.cat_data)

In [ ]:
models = list()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = config['cont_size']
hiddens_sizes = config['hidden_sizes']
output_size = 1

for i in range(config['nfolds']):
    model = Model(input_size,output_size,hiddens_sizes,embedding_sizes)
    model.load_state_dict(torch.load(f"./model{i}.bin",map_location=device))
    model.to(device)
    model.eval()
    models.append(model)

In [ ]:
def inference(test):
    all_prediction = np.zeros((test.shape[0],1))
    test_ds = TPSDataset(test,cat_features,cont_features)
    test_dl =  DataLoader(test_ds,batch_size = config["test_batch_size"],shuffle=False,num_workers = 4,pin_memory=True)
    
    for model in models:
        prediction = list()
        for inputs1,inputs2 in test_dl:
            inputs1 = inputs1.to(device)
            inputs2 = inputs2.to(device)
            outputs = model(inputs1,inputs2) 
            prediction.extend(outputs.detach().cpu().numpy())
        all_prediction += prediction

    return all_prediction/config['nfolds']

In [ ]:
predictions = inference(test_data)
sample.target = predictions
sample.to_csv('submission.csv',index=False)
sample.head()

In [ ]:
plt.figure(figsize=(15,7))
plt.subplot(131)
sns.distplot(sample.target)
plt.title("Distribution of test target")
plt.subplot(132)
sns.distplot(train_data[target_feature])
plt.title("Distribution of train target")
plt.subplot(133)
sns.distplot(sample.target,label='test')
sns.distplot(train_data[target_feature],label='train')
plt.legend()
plt.title("Distribution of train-test target");